In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import re


In [14]:
#look for the clean files
file_status = get_sed()
#filter out clean files and concat them
master_file = get_master_file()
#Create bigger lithologic group
sediment_classes_dict = denom_litho()
#lookup those bigger cat
master_file_denom = get_loc_denom(master_file)





AttributeError: 'DataFrame' object has no attribute 'LITHOLOGY'

In [162]:
loc_dest = 'C:\\Users\\mdulieu\\Documents\\hackforthesea\\Cod-Squad\\data\\sediment\\sediment_loc.csv'
master_file_denom.to_csv(loc_dest, index=False)

In [9]:
def get_sed_type(file):
    xl = pd.ExcelFile(file)
    xl = xl.parse()
    if 'LITHOLOGY' in xl:
        return 'good'        
    else:
        return 'bad'

In [10]:
def get_sed():
    loc_or = 'C:\\Users\\mdulieu\\Documents\\hackforthesea\\Cod-Squad\\data\\sediment\\raw\\'

    files = [f for f in listdir(loc_or) if isfile(join(loc_or, f))]
    
    file_status = pd.DataFrame([])
       
    for xl_file in files:
        sed = get_sed_type(loc_or + xl_file)
        file_status = file_status.append(pd.DataFrame({'file_name':xl_file,'status':sed}, index=[0]), ignore_index=True)
        
    return file_status


In [11]:
def get_master_file():
    loc_or = 'C:\\Users\\mdulieu\\Documents\\hackforthesea\\Cod-Squad\\data\\sediment\\raw\\'
    
    good_files_mask = file_status['status'] == 'good'

    good_files = file_status[good_files_mask]

    master_file = pd.DataFrame()

    for index, good_file in good_files.iterrows():
        file_path = loc_or + good_file.file_name
        xl = pd.ExcelFile(file_path)
        xl = xl.parse()
        master_file = master_file.append(xl[['LATITUDE', 'LONGITUDE', 'LITHOLOGY']])
        
    master_file = master_file.reset_index(drop=True)
        
    return master_file    

In [21]:
def denom_litho():
    sediment_classes = [
        'GRAVEL',
        'SAND',
        'CLAY',
        'MUD',
        'SILT',
        'CLAY',
        'ROCK',
        'SEDIMENT',
        'BEDROCK'
    ] 

    sediment_classes_dict = {}

    for sediment_class in sediment_classes:
        sediment_classes_dict[sediment_class] = []
        #looking at sand denominations
        sediment_denom_full = pd.DataFrame()
        for index, row in master_file.iterrows():
            try:
                if row.LITHOLOGY.endswith(sediment_class):
                    sediment_denom_full = sediment_denom_full.append(row)
            except:
                pass
            
        print(sediment_denom_full)

        sediment_classes_dict[sediment_class] = sediment_denom_full.LITHOLOGY.unique()

    return sediment_classes_dict

In [13]:
def get_loc_denom(master_file):
    master_file['LITHO_GROUP'] =pd.Series()

    for index, row in master_file.iterrows():
        for sediment_class in sediment_classes_dict:
            if row.LITHOLOGY in sediment_classes_dict[sediment_class]:
                master_file.loc[index,'LITHO_GROUP'] = sediment_class 
            else:
                pass

    return master_file


In [22]:
denom_litho()

        LATITUDE           LITHOLOGY  LONGITUDE
126    43.736747        SANDY GRAVEL -70.065872
135    43.630300  MUDDY SANDY GRAVEL -69.983011
136    43.591131  MUDDY SANDY GRAVEL -70.029128
137    43.557683  MUDDY SANDY GRAVEL -70.066731
144    43.539292  MUDDY SANDY GRAVEL -70.195617
145    43.558581              GRAVEL -70.225683
146    43.558789        SANDY GRAVEL -70.224069
157    43.726111        SANDY GRAVEL -69.887067
167    43.609175  MUDDY SANDY GRAVEL -69.816225
187    43.687586  MUDDY SANDY GRAVEL -69.927889
204    43.605906  MUDDY SANDY GRAVEL -69.973306
209    43.565525  SANDY MUDDY GRAVEL -69.985853
215    43.567958  MUDDY SANDY GRAVEL -70.144947
218    43.620219        SANDY GRAVEL -70.172217
222    43.673156  MUDDY SANDY GRAVEL -70.079364
223    43.648597  SANDY MUDDY GRAVEL -70.067817
225    43.642622  MUDDY SANDY GRAVEL -70.050639
226    43.651403  MUDDY SANDY GRAVEL -70.047028
231    43.687269  SANDY MUDDY GRAVEL -70.055100
241    43.542025        SANDY GRAVEL -70

        LATITUDE           LITHOLOGY  LONGITUDE
118    43.750753           SANDY MUD -70.137178
119    43.735867           SANDY MUD -70.145144
123    43.695575           SANDY MUD -70.113619
125    43.726794           SANDY MUD -70.077839
128    43.765422           SANDY MUD -70.088400
129    43.760872           SANDY MUD -70.104206
130    43.723753           SANDY MUD -70.153553
131    43.725453                 MUD -70.175986
133    43.673242           SANDY MUD -69.989222
134    43.660500           SANDY MUD -69.976981
139    43.527247           SANDY MUD -70.103881
150    43.612731           SANDY MUD -70.147650
153    43.700322           SANDY MUD -69.935453
164    43.616800                 MUD -69.767492
165    43.604011           SANDY MUD -69.782706
168    43.597981                 MUD -69.830408
169    43.585864                 MUD -69.844889
170    43.575061           SANDY MUD -69.859694
171    43.562558                 MUD -69.874578
174    43.579053                 MUD -69

       LATITUDE                               LITHOLOGY  LONGITUDE
3426   42.83333           OL.BRN.PEBLY SLT;IGNEOUS ROCK  -68.51667
3724   42.33333               DEEP DARK OLIVE CLAY;ROCK  -67.46667
3725   42.33333                    DARK OLIVE CLAY;ROCK  -67.33333
3749   42.28333                     SAND;SHELL AND ROCK  -65.63333
4368   42.88333           COARSE GRAVELLY SAND AND ROCK  -70.77000
4415   44.33667                           PROBABLY ROCK  -68.31850
4630   42.61667  ORANGE-BROWN PORPHYRITIC VOLCANIC ROCK  -70.66667
4631   42.61667    COARSE GRAINED GRANITIC LOOKING ROCK  -70.63167
4635   42.63167                           GRANITIC ROCK  -70.75167
4649   44.01167               BIOTITE AND GRANITIC ROCK  -69.13333
4654   44.40333                         QUARTZITIC ROCK  -68.22500
5390   42.60000               4 PEBBLES OF IGNEOUS ROCK  -70.13333
5403   42.92000                        PROBABLY BEDROCK  -70.75833
5421   44.32550                           PROBABLY ROCK  -68.3

{'BEDROCK': array(['PROBABLY BEDROCK', 'NO RECOVERY-PROBABLE BEDROCK',
        'NO RECOVERY-PROBABE BEDROCK'], dtype=object),
 'CLAY': array(['COARSE CLAY', 'CLAY', 'SAND-SILT-CLAY', 'SILTY CLAY', 'SILT-CLAY',
        'OLIVE SILT-CLAY', 'OLIVE BROWN SILT CLAY',
        'OLIVE BROWN SILT-CLAY', 'BROWN SILT CLAY', 'OLIVE SILT CLAY',
        'OLIVE BROWN AND BLACK GRAVELLY SAND-SILT CLAY',
        'OLIVE VERY FINE SAND;SILT-CLAY', 'OLIVE;STICKY SILT CLAY',
        'OLIVE SAND;SILT;CLAY', 'OLIVE SAND SILT CLAY',
        'OLIVE BROWN SAND;SILT;CLAY', 'LIGHT BROWN GRAY SILTY-CLAY',
        'LIGHT BROWN GRAY SAND-SILT-CLAY', 'PALE OLIVE SAND-SILT-CLAY',
        'GRAY BROWN SAND-SILT-CLAY', 'LIGHT OLIVE GRAY SILTY CLAY',
        'OLIVE GRAY SILT CLAY', 'OLIVE GRAY SANDY CLAY',
        'DARK BROWN SAND AND CLAY', 'DARK BROWN CLAY',
        'OLIVE FINE SAND AND CLAY', 'DARK OLIVE GRAY SILT CLAY',
        'DEEP OLIVE SILT-SAND AND CLAY', 'OLIVE GREEN CLAY',
        'BROWN VERY COARSE SAND WITH GR

In [157]:
master_file[~master_file['LITHO_GROUP'].isnull()]

,LATITUDE,LONGITUDE,LITHOLOGY,LITHO_GROUP
0,42.908778,-70.799447,SAND,SAND
1,42.909139,-70.791899,GRAVELLY SAND,SAND
2,42.908957,-70.784571,SAND,SAND
3,42.894315,-70.793509,GRAVELLY SAND,SAND
4,42.894328,-70.786274,MUDDY SAND,SAND
5,42.895366,-70.724210,SAND,SAND
6,42.879769,-70.806423,SAND,SAND
7,42.879868,-70.798990,MUDDY SAND,SAND
8,42.879969,-70.791447,SAND,SAND
9,42.880070,-70.783904,SAND,SAND
